In [ ]:
!pip3 install --upgrade pip
!pip3 install -r requirements.txt

## Example 1: Convert Keras App Model to OpenVINO IR

In [ ]:
from ov_utils import create_ir
import os

### Provide below details:

#### model name to the variable "keras_app_model_name"
#### S3 bucket name with "sagemaker" as part of the name. (important for Sagemaker access) to the variable "bucket_name". Ex: sagemaker-ir-creation
#### precision_type can be "FP32" or "FP16".

#### NOTE: If the bucket does not exist then it will be created.

In [ ]:
keras_app_model_name = "Your_model_name"
bucket_name = "Your_bucket_name"
precision_type = "Your_precision"

#### Create IR params.

In [ ]:
output_dir = "".join(["./", keras_app_model_name, "-keras-", precision_type])
model_name = "".join([keras_app_model_name])
create_ir_params = {
    "keras_app_model_name": keras_app_model_name,
    "output_dir": output_dir,
    "mo_params": {"data_type": precision_type, "model_name": model_name},
    "bucket_name": bucket_name
}

### Start the IR creation and upload to S3.

In [ ]:
try:
    create_ir(create_ir_params)
    print (f"IR files created for model:{model_name} and the same is uploaded in S3:{bucket_name}")
except Exception as err:
    print(f"FAILED: Please find the error details below:")
    print(err)